# Counting residues


MIToS was designed to perform fast counting of residues. To achieve that,
each `Residue` is encoded as an integer that can be used to index.

In [ ]:
using MIToS.MSA
res = Residue('C')

In [ ]:
Int(res)

Let's count residues in the *UBA domain* Pfam family.

In [ ]:
using MIToS.Pfam
const msa_file = downloadpfam("PF16577")

In [ ]:
const msa = read(msa_file, Stockholm)

We can access the MSA as a matrix to get a particular column or sequence:
```julia
msa[:, column]
msa[sequence, :]
```

In [ ]:
col_i = msa[:, 10]

MIToS Information module has functions to get residue frequencies (counts or
fractions) and to return contingency tables:

In [ ]:
using MIToS.Information

In [ ]:
Ni = count(col_i)

In [ ]:
Pi = probabilities(col_i)

`Counts` and `Probabilities` objects wrap a `ContingencyTable`:

In [ ]:
Pi.table

They can be quickly indexed by using `Residue`s:

In [ ]:
Pi[Residue('P')]

You can create a bidimensional `ContingencyTable` by using two MSA columns or
sequences (there is no limit to the number of dimensions):

In [ ]:
col_j = msa[:, 15]

Nij = count(col_i, col_j)

In [ ]:
Nij[Residue('P'), Residue('R')]

You can also get probabilities by normalizing counts:

In [ ]:
Pij = normalize(Nij.table)

In [ ]:
Pij[Residue('P'), Residue('R')]

## Plotting counts

We use a `heatmap` to plot a bidimensional table:

In [ ]:
using Plots
gr()

We can list the available color libraries:

In [ ]:
clibraries()

Visualize the color maps defined in a library:

In [ ]:
showlibrary(:Plots)

And select the color library we want to use:

In [ ]:
clibrary(:cmocean)

For this plot, we use the interactive *Plotly* backend

In [ ]:
plotlyjs()

We need a vector of strings for the ticks labels...

In [ ]:
x = string.(Residue.(UngappedAlphabet()))

...because `xticks` and `xticks` keyword arguments take a tuple:
`(positions, labels)`

In [ ]:
heatmap(Nij,
		color=:tempo,
		ratio=:equal,
		xticks=(1:length(x), x),
		yticks=(1:length(x), x))

## Functions taking counts and probabilities

Information defines different functions that take contingency tables as an
input, for example:

In [ ]:
entropy(Ni)

In [ ]:
mutual_information(Nij)

### Conservation

You can use the MIToS' Information module to calculate the Shannon entropy of
each MSA column:

In [ ]:
using MIToS.Information

const alphabet = UngappedAlphabet()

You also have `GappedAlphabet` and `ReducedAlphabet`:

In [ ]:
ReducedAlphabet("(AILMV)(RHK)(NQST)(DE)(FWY)CGP")

Empty contingency tables can be defined by using
`ContingencyTable(element_type, Val{dimensions}, alphabet)`:

In [ ]:
const table = ContingencyTable(Int, Val{1}, alphabet)

The `mapcolfreq!`, `mapseqfreq!`, `mapcolpairfreq!` and `mapseqpairfreq!`
functions from the Information module apply a function on the table filled
using each column, sequence, column pair or sequence pair of the MSA,
respectively:

In [ ]:
Hx = mapcolfreq!(entropy, msa, Counts(table))

We use **Plots.jl** to visualize the entropy of each column to find variable
and conserved regions

In [ ]:
gr()

In [ ]:
plot_entropy = plot(vec(Hx),
					color=:orange,
					fill=0,
					fillalpha=0.5,
					legend=false,
					ylab="Entropy")

In [ ]:
plot_msa = plot(msa, legend=false)

In [ ]:
plot(plot_entropy,
	 plot_msa,
	 layout=grid(2, 1, heights=[0.2, 0.8]),
	 link=:x)

## Multivariate mutual information

We are going to measure mutual information in MSA column triplets. We use
Julia parallelism because the number of combinations is high.

In [ ]:
using Distributed

In [ ]:
nprocs()

In [ ]:
addprocs(3)

In [ ]:
nprocs()

We are going to use [Combinatorics](https://github.com/JuliaMath/Combinatorics.jl)
everywhere (in all the process) to get a generator of the combinations and
[ProgressMeter](https://github.com/timholy/ProgressMeter.jl) to see the
progress of the parallel loop.

In [ ]:
@everywhere using Combinatorics
@everywhere using ProgressMeter
@everywhere using MIToS.MSA
@everywhere using MIToS.Information

#### Exercise 1

Fill the missing parts of the function to calculate mutual information
between MSA columns i, j and k.

In [ ]:
function parallel_mmi(msa)
	ncols = ncolumns(msa)
	triplets = combinations(1:ncols, 3)
	@showprogress pmap(triplets) do (i, j, k)
#           ...to complete...
#           (i, j, k) => ...to complete...
	end
end

In [ ]:
mi_values = parallel_mmi(msa)

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*